In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
import pandas as pd


2026-01-30 10:40:13.571474: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-30 10:40:14.446432: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-30 10:40:16.128841: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-4

In [3]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

2026-01-30 10:40:44.234177: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/home/legramante/Documentos/cat-and-dogs_Machine-Learning/cats_and_dogs/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/home/legramante/Documentos/cat-and-dogs_Machine-Learning/cats_and_dogs/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/home/legramante/Documentos/cat-and-dogs_Machine-Learning/cats_and_dogs/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

Found 17500 files belonging to 2 classes.
Found 3750 files belonging to 2 classes.
Found 3750 files belonging to 2 classes.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

In [6]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

In [7]:
inputs = tf.keras.Input(shape=(224, 224, 3))

x = data_augmentation(inputs)
x = preprocess_input(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

In [8]:
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [9]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 710s 646ms/step - accuracy: 0.9693 - loss: 0.0809 - val_accuracy: 0.9891 - val_loss: 0.0284
Epoch 2/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 702s 642ms/step - accuracy: 0.9850 - loss: 0.0430 - val_accuracy: 0.9899 - val_loss: 0.0282
Epoch 3/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 782s 715ms/step - accuracy: 0.9868 - loss: 0.0359 - val_accuracy: 0.9899 - val_loss: 0.0251
Epoch 4/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 842s 770ms/step - accuracy: 0.9879 - loss: 0.0327 - val_accuracy: 0.9877 - val_loss: 0.0330
Epoch 5/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 748s 683ms/step - accuracy: 0.9891 - loss: 0.0291 - val_accuracy: 0.9907 - val_loss: 0.0278
Epoch 6/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 685s 626ms/step - accuracy: 0.9903 - loss: 0.0258 - val_accuracy: 0.9904 - val_loss: 0.0247
Epoch 7/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 682s 623ms/step - accuracy: 0.9900 - loss: 0.0260 - val_accuracy: 0.9896 - val_loss: 0.0275
Epoch 8/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 717s 655ms/step - ac

In [10]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Acurácia no teste: {test_acc:.4f}")


235/235 ━━━━━━━━━━━━━━━━━━━━ 129s 548ms/step - accuracy: 0.9896 - loss: 0.0305
Acurácia no teste: 0.9896


In [12]:
history_df = pd.DataFrame(history.history)

history_df["experiment"] = "resnet50_transfer_learning"
history_df["augmentation"] = "yes"
history_df["fine_tuning"] = "no"
history_df["learning_rate"] = LEARNING_RATE
history_df["notes"] = "ResNet50 com camadas congeladas e LR baixo"

history_df.to_csv(
    "/home/legramante/Documentos/cat-and-dogs_Machine-Learning/experiments/resnet50_results.csv",
    index=False
)


In [14]:
model.save("/home/legramante/Documentos/cat-and-dogs_Machine-Learning/models/resnet50_transfer_learning.keras")